# PHYS20762 Computational Physics
## Project 3: Monte Carlo Techniques
### Neutron Transport and Scattering Through a Shielding Layer

Author: David Phelan  
The University of Manchester  
Date: May 2025  

---

## 1. Introduction

In this project, we use Monte Carlo techniques to simulate the behaviour of thermal neutrons passing through various shielding materials. The aim is to model neutron scattering, absorption, and transmission processes within slabs of water, lead, and graphite.

By generating random numbers, random points in three dimensions, and implementing exponential distributions, we build up to a full neutron transport simulation. We will visualise random walks, quantify transmission, reflection, and absorption rates, and determine the characteristic attenuation lengths for different materials.

---

### 1.1 Objectives

- Verify the generation of uniform random numbers using `numpy.random.uniform()`.
- Generate and visualise random 3D points to assess distribution properties.
- Create and test an exponential random number generator.
- Produce isotropic random directions and isotropic steps with exponentially distributed lengths.
- Simulate neutron random walks through slabs of different materials.
- Quantify neutron absorption, reflection, and transmission rates as functions of slab thickness.
- Determine attenuation lengths from transmission data.
- (Optional) Implement the Woodcock method to simulate transport through two adjacent materials.

Throughout the project, we aim to support the simulation with appropriate plots and discuss the numerical results, including any uncertainties.

---

## 2. Setup & Initialization
Before numerically solving the equations of motion, we first set up the computational environment by:  
- Installing and importing essential libraries.  
- Ensuring all dependencies are available before proceeding.  

#### Imported Libraries
- `numpy` – for numerical computations and array handling.  
- `matplotlib.pyplot` – for plotting and visualizing results.  
- `cmath` – for handling complex numbers in analytical solutions.  

These libraries provide the necessary tools to implement and analyze the numerical methods effectively.

In [ ]:
# Install required packages (only run this (once) if needed)
%pip install numpy
%pip install scipy
%pip install plotly
%pip install collections
%pip install time
%pip install os
%pip install shutil

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import plotly.graph_objects as go
from collections import Counter
import time
import os
import shutil

Seed setting

In [ ]:
np.random.seed(42)  # For reproducibility

Intro assignment

Intro classes

In [ ]:
class Material:
    BARN_TO_CM2 = 1e-24  # cm²
    AVOGADRO_CONSTANT = 6.02214076e23  # 1/mol

    def __init__(self, name, scattering_cross_section, absorption_cross_section):
        self.name = name
        self.total_cross_section = scattering_cross_section + absorption_cross_section
        self.scattering_cross_section = scattering_cross_section
        self.absorption_cross_section = absorption_cross_section
        self.absorption_probability = absorption_cross_section / self.total_cross_section

    @staticmethod
    def construct_from_microscopic_properties(name, sigma_a, sigma_s, density, molar_mass):
        number_density = Material.AVOGADRO_CONSTANT * density / molar_mass

        return Material(
            name,
            number_density * sigma_s * Material.BARN_TO_CM2,
            number_density * sigma_a * Material.BARN_TO_CM2
        )
    
class Slab:
    def __init__(self, material, width):
        self.material = material
        self.width = width

Intro initalising classes

In [ ]:
# Define materials with their properties
water = Material.construct_from_microscopic_properties(
    "Water", 0.6652, 103.0, 1.0, 18.0153)

lead = Material.construct_from_microscopic_properties(
    "Lead", 0.158, 11.221, 11.35, 207.2)

graphite = Material.construct_from_microscopic_properties(
    "Graphite", 0.0045, 4.74, 1.67, 12.011)

materials = [water, lead, graphite]

rest of blah

In [ ]:
plot_width = 1920/2
plot_height = 1080/2

fig1 = go.Figure(
    data=[go.Histogram(x=np.random.uniform(0, 1, 1000), nbinsx=20)],
    layout=dict(
        width=plot_width,  # width in pixels
        height=plot_height,  # height in pixels
        title="Histogram",
        xaxis_title="Value",
        yaxis_title="Count"
    )
)
fig1.show()

z = np.random.uniform(0, 1, 1000)
fig2 = px.scatter_3d(
    x=np.random.uniform(0, 1, 1000), y=np.random.uniform(0, 1, 1000), z=z, color=z,
    labels={'x': 'X Axis', 'y': 'Y Axis', 'z': 'Z Axis'}
)
fig2.update_traces(marker=dict(size=2))  # Smaller points
fig2.update_layout(
    title="3D Scatter Plot",
    width=plot_width,   # width in pixels
    height=plot_height   # height in pixels
)
fig2.show()

Bad one:
watch random num gen video for info (phsyics) about why it's bad

In [ ]:
class RandSSP:
    def __init__(self, seed=123456789):
        self.m = 2**31
        self.a = 2**16 + 3
        self.c = 0
        self.x = seed

    def generate(self, p, q):
        r = np.zeros((p, q))
        for l in range(q):
            for k in range(p):
                self.x = (self.a * self.x + self.c) % self.m
                r[k, l] = self.x / self.m
        return r

# Create generator instance
generator = RandSSP()

# Generate 3D data
r = generator.generate(3, 1500)
mydatax, mydatay, mydataz = r[0, :], r[1, :], r[2, :]

# Create 2D histogram
fig3 = go.Figure(
    data=[go.Histogram(x=mydatax, nbinsx=20)],
    layout=dict(
        width=plot_width,  # width in pixels
        height=plot_height,  # height in pixels
        title="Histogram",
        xaxis_title="Value",
        yaxis_title="Count"
    )
)
fig3.show()

# Create 3D scatter plot
fig4 = px.scatter_3d(
    x=mydatax, y=mydatay, z=mydataz, color=mydataz,
    labels={'x': 'X Axis', 'y': 'Y Axis', 'z': 'Z Axis'}
)
fig4.update_traces(marker=dict(size=2))  # Smaller points
fig4.update_layout(
    title="3D Scatter Plot",
    width=plot_width,
    height=plot_height
)
fig4.show()


In [ ]:
for material in materials:
    samples = 1000
    macroscopic_cross_section= material.absorption_cross_section
    uniform_random_numbers = np.random.uniform(0, 1, samples)

    # Generate the exponential distribution
    exponential_random_numbers = - np.log(uniform_random_numbers) / macroscopic_cross_section

    # Bin the data
    counts, bin_edges = np.histogram(exponential_random_numbers, bins=100, range=(0, max(exponential_random_numbers)))

    # Compute bin centres (for plotting and fitting, so we don't calculate the bins twice)
    # Bin edges are the boundaries of the bins, so we need to average them to get the centres
    bin_left_edges = np.array([left_edge for left_edge in bin_edges[:-1]])
    # Compute the survival function: cumulative sum from right to left
    cumulative_counts = np.cumsum(counts[::-1])[::-1]

    # Create histogram trace using binned data
    histogram_trace = go.Scatter(
        x=bin_left_edges,
        y=cumulative_counts,
        mode='markers',
        name=f"Simulated survival function, {material.name}"
    )

    max_distance = max(exponential_random_numbers)
    theoretical_x = np.linspace(0, max_distance, samples)
    theoretical_y = samples * np.exp(-theoretical_x * macroscopic_cross_section)

    theoretical_trace = go.Scatter(
        x=theoretical_x,
        y=theoretical_y,
        mode='lines',
        name=f"Theoretical survival function, {material.name}"
    )

    line_trace = go.Scatter(
        x=[0, max_distance],
        y=[samples / np.e] * 2,
        mode='lines',
        line=dict(color='red', dash='dash'),
        name=f"{material.name} - N_0/e Line"
    )

    # Combine all traces and plot
    layout = go.Layout(
        title=f"Survival function for {material.name}",
        xaxis=dict(title="Distance (cm)"),
        yaxis=dict(title="Counts"),
        legend=dict(x=0, y=1)
    )

    fig = go.Figure(
        data=[histogram_trace, theoretical_trace, line_trace], layout=layout)
    fig.update_layout(
        legend=dict(
            x=1,
            y=1,
            xanchor="left",
            yanchor="top"
        )
    )
    fig.show()

    x2 = bin_left_edges[cumulative_counts > 0]
    y2 = np.log(cumulative_counts[cumulative_counts > 0])

    weights = np.sqrt(cumulative_counts[cumulative_counts > 0])

    fit, cov = np.polyfit(x2, y2, 1, cov=True)

    slope = fit[0]
    slope_uncertainty = np.sqrt(cov[0][0])
    simulated_attenuation_length = -1 / slope
    uncertainty = slope_uncertainty / slope**2

    print(f"Simulated attenuation length, using binned, for {material.name}: {simulated_attenuation_length:.2f} ± {uncertainty:.2f}")
    print(f"Theoretical attenuation length for {material.name}: {1/macroscopic_cross_section:.2f}")

    """
    In accordance with the assignment instructions, the first figure displays a binned histogram of neutron distances
    with an overlaid theoretical exponential curve. While broadly consistent, the simulated data shows a slight rightward
    bias due to binning: counts are grouped at bin midpoints, whereas the underlying exponential decay is continuous.

    To address this, a second plot uses ranked raw data and plots the natural logarithm of the number of neutrons that
    reached at least a given distance. This approach eliminates binning error and results in a significantly closer match
    to the theoretical decay curve, as shown by the alignment between the simulated and expected values. The improved
    accuracy is also reflected in the fitted attenuation length, which closely matches the theoretical prediction.
    """

    sorted_data = np.sort(exponential_random_numbers)
    ranks = np.arange(len(sorted_data), 0, -1)  # from N to 1

    fit, cov = np.polyfit(sorted_data, np.log(ranks), 1, cov=True)

    slope = fit[0]
    slope_uncertainty = np.sqrt(cov[0][0])
    simulated_attenuation_length = -1 / slope
    uncertainty = slope_uncertainty / slope**2
    
    print(f"Simulated attenuation length, using ranked, for {material.name}: {simulated_attenuation_length:.2f} ± {uncertainty:.2f}")
    print(f"Theoretical attenuation length for {material.name}: {1/macroscopic_cross_section:.2f}")

Need to put old version in where he use numpy histogram and show that it leads to bad lambda because everything is right shifted (binning an exponential, which is left-side heavy, taking the midpoint shifts the majority of the data to the right)
And isn't accurate because we want "how deep did things go" and so it makes things sound like they went deeper than they did
So we do this version where we ...

Week Two musbeans

In [ ]:
def generate_random_unit_vector():

    theta = np.arccos(np.random.uniform(-1, 1))
    phi = np.random.uniform(0, 2*np.pi)

    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)

    print
    return x, y, z


x,y,z = zip(*[(x,y,z) for x,y,z in [generate_random_unit_vector() for _ in range(10000)]])


# Create 3D scatter plot
fig6 = px.scatter_3d(
    x=x, y=y, z=z, color=z,
    labels={'x': 'X Axis', 'y': 'Y Axis', 'z': 'Z Axis'}
)
fig6.update_traces(marker=dict(size=2))  # Smaller points
fig6.update_layout(
    title="3D Scatter Plot",
    width=plot_width,
    height=plot_height
)
fig6.show()

pesky poles - discuss physics (if it's in scope (too detailed)) of why randomly generating theta is bad
video says:
what I thought was the reason as to why the problem should be happening - middle of sphere is much larger than poles therefore points spread out (I think)

next point


In [ ]:
def random_uniform_path_length(cross_section):
    return -np.log(np.random.uniform(0, 1)) / cross_section

def random_walk(cross_section):
    x, y, z = (0, 0, 0)
    yield (x, y, z)
    while True:
        step_size = random_uniform_path_length(cross_section)
        dx, dy, dz = generate_random_unit_vector()
        x += step_size * dx
        y += step_size * dy
        z += step_size * dz
        yield (x, y, z)

for material in materials:
    macroscopic_cross_section= material.absorption_cross_section
    
    positions_iterator = random_walk(macroscopic_cross_section)
    positions = [next(positions_iterator) for _ in range(100)]

    x, y, z = zip(*positions)
    scatter = go.Scatter3d(x=x, y=y, z=z, mode='lines', marker=dict(size=2))
    layout = go.Layout(
        title=f"Random Walk in {material.name}",
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        )
    )
    fig = go.Figure(data=[scatter], layout=layout)
    fig.show()

    # Do some random walks and plot the results

Week 2 done. Onto rest of assignment

In [ ]:
def random_walk_slab(cross_section, slab_thickness, absorption_probability):
    x, y, z = (0, 0, 0)
    yield (x, y, z)
    dx, dy, dz = (1, 0, 0)  # Initial direction
    while True:
        step_size = random_uniform_path_length(cross_section)
        x += step_size * dx
        y += step_size * dy
        z += step_size * dz
        if np.random.uniform(0, 1) < absorption_probability:
            print("Absorption occurred")
            break
        elif x > slab_thickness:
            print("Exited the slab")
            break  # Exit the slab
        elif x < 0:
            print("Reflected at the boundary")
            # Reflect at the boundary
            break
        else:
            # Scattering occurs
            dx, dy, dz = generate_random_unit_vector()
        yield (x, y, z)

for material in materials:
    print(f"{material.name}: {material.absorption_probability:.5f}")

for material in materials:
    
    positions_iterator = random_walk_slab(material.total_cross_section, 10, material.absorption_probability)
    positions = list(positions_iterator)

    x, y, z = zip(*positions)
    scatter = go.Scatter3d(x=x, y=y, z=z, mode='lines', marker=dict(size=2))
    layout = go.Layout(
        title=f"Random Walk in {material.name}",
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        )
    )
    fig = go.Figure(data=[scatter], layout=layout)
    fig.show()

    # Do some random walks and plot the results

In [ ]:
def random_walk_slab(cross_section, slab_thickness, absorption_probability):
    result = []
    x, y, z = (0, 0, 0)
    result.append((x, y, z))
    dx, dy, dz = (1, 0, 0)  # Initial direction
    while True:
        step_size = random_uniform_path_length(cross_section)
        x += step_size * dx
        y += step_size * dy
        z += step_size * dz
        if np.random.uniform(0, 1) < absorption_probability:
            return result, "absorbed"
        elif x > slab_thickness:
            # Exit the slab
            return result, "transmitted"
        elif x < 0:
            # Reflect at the boundary
            return result, "reflected"
        else:
            # Scattering occurs
            dx, dy, dz = generate_random_unit_vector()
        result.append((x, y, z))


colours = {
    "absorbed": "red",
    "transmitted": "blue",
    "reflected": "green"
}

for material in materials:
    layout = go.Layout(
        title=f"Random Walk in {material.name}",
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        )
    )
    fig = go.Figure(layout=layout)
    for i in range(100):
        positions, fate = random_walk_slab(material.total_cross_section, 3, material.absorption_probability)
        x, y, z = zip(*positions)
        scatter = go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(color=colours[fate]), marker=dict(size=2))
        fig.add_trace(scatter)
    
    fig.show()

    # Do some random walks and plot the results

In [ ]:
for material in materials:
    layout = go.Layout(
        title=f"Random Walk Outcomes vs Slab Thickness for {material.name}",
        scene=dict(
            xaxis=dict(title='Slab Thickness'),
            yaxis=dict(title='Count'),
        )
    )

    fig2d = go.Figure(layout=layout)

    thicknesses = []
    N_A_list = []
    N_R_list = []
    N_T_list = []

    for thickness in range(20):

        N = 1000  # Number of random walks
        N_A = 0
        N_R = 0
        N_T = 0

        for i in range(N):
            _, fate = random_walk_slab(material.total_cross_section, thickness, material.absorption_probability)
            if fate == "absorbed":
                N_A += 1
            elif fate == "transmitted":
                N_T += 1
            elif fate == "reflected":
                N_R += 1
                
        thicknesses.append(thickness)
        N_A_list.append(N_A)
        N_R_list.append(N_R)
        N_T_list.append(N_T)


    
    N_A_array = np.array(N_A_list)
    N_R_array = np.array(N_R_list)
    N_T_array = np.array(N_T_list)

    p_A = N_A_array / N
    p_A_unc = np.sqrt(p_A*(1 - p_A)/N)
    p_R = N_R_array / N
    p_R_unc = np.sqrt(p_R*(1 - p_R)/N)
    p_T = N_T_array / N
    p_T_unc = np.sqrt(p_T*(1 - p_T)/N)

    fig2d.add_trace(go.Scatter(
        x=thicknesses,
        y=p_A,
        mode='markers',
        name='Absorbed',
        error_y=dict(type='data', array=p_A_unc, visible=True)
    ))
    fig2d.add_trace(go.Scatter(
        x=thicknesses,
        y=p_R,
        mode='markers',
        name='Reflected',
        error_y=dict(type='data', array=p_R_unc, visible=True)
    ))
    fig2d.add_trace(go.Scatter(
        x=thicknesses,
        y=p_T,
        mode='markers',
        name='Transmitted',
        error_y=dict(type='data', array=p_T_unc, visible=True)
    ))
    fig2d.show()

In [ ]:
def random_walk_double_slab(slab_thickness, first_slab, second_slab):
    result = []
    x, y, z = (0, 0, 0)
    result.append((x, y, z))
    dx, dy, dz = (1, 0, 0)  # Initial direction

    acceptance_probability = min(first_slab.total_cross_section, second_slab.total_cross_section) / max(first_slab.total_cross_section, second_slab.total_cross_section)
    while True:
        
        if first_slab.total_cross_section < second_slab.total_cross_section:

            step_size = random_uniform_path_length(second_slab.total_cross_section)
            x += step_size * dx
            y += step_size * dy
            z += step_size * dz

            if x <= slab_thickness:
                if np.random.uniform(0, 1) < acceptance_probability:
                    if np.random.uniform(0, 1) < first_slab.absorption_probability:
                        return result, "absorbed"
                    elif x < 0:
                        return result, "reflected"
                    else:
                        # Scattering occurs
                        dx, dy, dz = generate_random_unit_vector()
                else:
                    # Scattering occurs
                    dx, dy, dz = generate_random_unit_vector()

                result.append((x, y, z))

            else:
                if np.random.uniform(0, 1) < first_slab.absorption_probability:
                    return result, "absorbed"
                elif x > 2 * slab_thickness:
                    return result, "transmitted"
                else:
                    dx, dy, dz = generate_random_unit_vector()

                result.append((x, y, z))

        elif first_slab.total_cross_section > second_slab.total_cross_section:
           
            step_size = random_uniform_path_length(first_slab.total_cross_section)
            x += step_size * dx
            y += step_size * dy
            z += step_size * dz

            if x <= slab_thickness:
                if np.random.uniform(0, 1) < first_slab.absorption_probability:
                    return result, "absorbed"
                elif x < 0:
                    return result, "reflected"
                else:
                    # Scattering occurs
                    dx, dy, dz = generate_random_unit_vector()

                result.append((x, y, z))
                
            else:
                if np.random.uniform(0, 1) < acceptance_probability:
                    if np.random.uniform(0, 1) < first_slab.absorption_probability:
                        return result, "absorbed"
                    elif x > 2 * slab_thickness:
                        return result, "transmitted"
                    else:
                        dx, dy, dz = generate_random_unit_vector()

                result.append((x, y, z))

N_A = 0
N_R = 0
N_T = 0

for i in range(1000):
    _, fate = random_walk_double_slab(1, water, graphite)
    if fate == "absorbed":
        N_A += 1
    elif fate == "transmitted":
        N_T += 1
    elif fate == "reflected":
        N_R += 1
    
print(f"Absorbed: {N_A}, Transmitted: {N_T}, Reflected: {N_R}")



N_A = 0
N_R = 0
N_T = 0

for i in range(1000):
    _, fate = random_walk_slab(water.total_cross_section, 1, water.absorption_probability)
    if fate == "absorbed":
        N_A += 1
    elif fate == "transmitted":
        N_T += 1
    elif fate == "reflected":
        N_R += 1

N_X = 0
N_Y = 0
N_Z = 0

TEST = N_T
for i in range(TEST):
    _, fate = random_walk_slab(graphite.total_cross_section, 1, graphite.absorption_probability)
    if fate == "absorbed":
        N_X += 1
    elif fate == "transmitted":
        N_Y += 1
    elif fate == "reflected":
        N_Z += 1

print(f"Absorbed: {N_X}, Transmitted: {N_Y}, Reflected: {N_Z}")
print(N_A + N_R + N_X + N_Y + N_Z)